# Vertex Pipeline with KFP (Kube Flow Pipelines)

In [ ]:
!pip install google-cloud-aiplatform==1.50.0 kfp==2.7.0

In [ ]:
!pip install google-cloud-pipeline-components==2.14.0

In [ ]:
YOUR_NAME="your-name" # TO-DO: store your name in this var

In [ ]:
PROJECT_ID='mlops-421501'
BUCKET=f"gs://{PROJECT_ID}-{YOUR_NAME}-bucket"

In [ ]:
import kfp

from kfp.v2 import compiler, dsl
from kfp.v2.dsl import component, pipeline, Artifact, ClassificationMetrics, Input, Output, Model, Metrics

from google.cloud import aiplatform
import google_cloud_pipeline_components
from typing import NamedTuple

In [ ]:
PATH=%env PATH
%env PATH={PATH}:/home/jupyter/.local/bin
REGION="us-central1"

PIPELINE_ROOT = f"{BUCKET}/pipeline_root/"
PIPELINE_ROOT

In [ ]:
@component(base_image="python:3.9")
def product_name(text: str) -> str:
    return text

In [ ]:
@component(base_image="python:3.8", packages_to_install=["emoji"])
def emoji(
    text: str,
) -> NamedTuple(
    "Outputs",
    [
        ("emoji_text", str),  # Return parameters
        ("emoji", str),
    ],
):
    import emoji

    emoji_text = text
    emoji_str = emoji.emojize(':' + emoji_text + ':', language='alias')
    print("output one: {}; output_two: {}".format(emoji_text, emoji_str))
    return (emoji_text, emoji_str)

In [ ]:
@component
def build_sentence(
    product: str,
    emoji: str,
    emojitext: str
) -> str:
    print("We completed the pipeline, hooray!")
    end_str = product + " is "
    if len(emoji) > 0:
        end_str += emoji
    else:
        end_str += emojitext
    return(end_str)

In [ ]:
@pipeline(
    name="hello-world",
    description="An intro pipeline",
    pipeline_root=PIPELINE_ROOT,
)

# You can change the `text` and `emoji_str` parameters here to update the pipeline output
def intro_pipeline(
    text: str = "Vertex Pipelines", 
    emoji_str: str = "sparkles"
):
    product_task = product_name(text=text)
    emoji_task = emoji(text=emoji_str)
    consumer_task = build_sentence(
        product=product_task.output,
        emoji=emoji_task.outputs["emoji"],
        emojitext=emoji_task.outputs["emoji_text"],
    )

In [ ]:
compiler.Compiler().compile(
    pipeline_func=intro_pipeline, package_path="intro_pipeline_job.json"
)

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

In [ ]:
job = aiplatform.PipelineJob(
    display_name=f"{YOUR_NAME}-hello-world-pipeline",
    template_path="intro_pipeline_job.json",
    job_id="hello-world-pipeline-{0}".format(TIMESTAMP),
    enable_caching=True
)

In [ ]:
job.submit()